# RAG Implementation

In [18]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
load_dotenv()

True

In [19]:
llm = ChatGroq(model_name="Llama3-8b-8192")

# llm = ChatOllama(
#     model = "deepseek-r1:1.5b",
#     temperature = 0,
#     num_predict = 256,
#     # other params ...
# )


In [20]:
def embd_load_vectordb(filepath, vectordb_path):
    # Initialize the embedding model
    embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        # Load and split the PDF document
    docs = PyPDFLoader(filepath).load_and_split()
    # Create a Chroma vector store with a specified directory for persistence
    vectordb = Chroma.from_documents(docs, embedding, persist_directory=vectordb_path)
    print("Vector database created and persisted.")
    return vectordb


In [21]:
def load_vectordb(vectordb_path):
    embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    loaded_db = Chroma(persist_directory=vectordb_path, embedding_function=embedding)
    return loaded_db

In [22]:
def response_generator(vectordb, query, llm):

    template = """
    You are an intelligent assistant designed to provide accurate and concise answers based on the context provided. 
    Follow these rules strictly:
    1. Use ONLY the information provided in the context to answer the question.
    2. If the context does not contain enough information to answer the question, say "I don't know."
    3. Do not make up or assume any information outside of the context.
    4. Keep your answer concise and to the point (maximum 3 sentences).

    Context:
    {context}

    Question:
    {question}

    Helpful Answer:
    """


    QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

    qa_chain = RetrievalQA.from_chain_type(llm, 
                                           retriever=vectordb.as_retriever(search_kwargs={"k": 3}), 
                                           return_source_documents=True, 
                                           chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

    ans = qa_chain.invoke(query)
    return ans["result"]


In [26]:
vectordb_path = "./test_db"
data_path = "./knowledge/mitsubisi_electric.pdf"
# C:\Users\mayur\Desktop\datasmith ai\knowledge\mitsubisi_electric.pdf

In [27]:
# emed_data = embd_load_vectordb("./knowledge/health_products_data.pdf")
emed_data = embd_load_vectordb(data_path, vectordb_path)

vectordb = load_vectordb(vectordb_path)

Vector database created and persisted.


In [32]:
query = "what is special stop input?"
answer = response_generator(vectordb, query, llm)
print(answer)

Based on the provided context, the special stop input is an "emergency stop input" which detects emergency stop inputs and makes the STO function meet the requirements of SIL 3, Category 4, PL e.


## Loader using PDF miner

In [37]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.pdfpage import PDFPage


In [39]:
def get_pdf_file_content(path_to_pdf):
    
    '''
    path_to_pdf: is the parameter that will give access to the PDF File 
    we want to extract the content.
    '''
    
    '''
    PDFResourceManager is used to store shared resources such as fonts or images that 
    we might encounter in the files. 
    '''
    
    resource_manager = PDFResourceManager(caching=True)
    
    '''
    create a string object that will contain the final text the representation of the pdf. 
    '''
    out_text = StringIO()
    
    '''
    UTF-8 is one of the most commonly used encodings, and Python often defaults to using it.
    In our case, we are going to specify in order to avoid some encoding errors.
    '''
    codec = 'utf-8'
    
    """
    LAParams is the object containing the Layout parameters with a certain default value. 
    """
    laParams = LAParams()
    '''
    Create a TextConverter Object, taking :
    - ressource_manager,
    - out_text 
    - layout parameters.
    '''
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    
    '''
    Create a PDF interpreter object taking: 
    - ressource_manager 
    - text_converter
    '''
    interpreter = PDFPageInterpreter(resource_manager, text_converter)

    '''
    We are going to process the content of each page of the original PDF File
    '''
    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)

        '''
    Retrieve the entire contents of the “file” at any time 
    before the StringIO object’s close() method is called.
    '''
    text = out_text.getvalue()

    '''
    Closing all the ressources we previously opened
    '''
    fp.close()
    text_converter.close()
    out_text.close()
    
    '''
    Return the final variable containing all the text of the PDF
    '''
    return text

In [40]:
path_to_pdf = "./knowledge/mitsubisi_electric.pdf"

In [41]:
print(get_pdf_file_content(path_to_pdf))

RV-FR Series Standard Specifications Manual 
(RV-2/4/7/13/20FR Series)

RV-2FR series
RV-4FR series
RV-7FR series
RV-13FR series
RV-20FR series

SAFETY PRECAUTIONS

Read the separate Safety Manual thoroughly before using this product. Take precautions as necessary.

Before using this product, please read this manual and the relevant manuals carefully and pay full attention to safety to handle 

the product correctly.

The precautions given in this manual are concerned with this product only. For the safety precautions of the robot system, 

refer to the user's manual for the peripheral equipment used.

In this manual, the safety precautions are classified into three levels: "

DANGER", " WARNING", and "

CAUTION".

Precaution indicating cases where there is a risk of operator fatality or
serious injury if handling is mistaken. Always observe these precautions
to safely use the robot.

Precaution indicating cases where the operator could be subject to
fatalities or serious injuries if

In [46]:
import os
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

def get_pdf_file_content(path_to_pdf, output_md_file):
    resource_manager = PDFResourceManager(caching=True)
    out_text = StringIO()
    laParams = LAParams()
    
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    interpreter = PDFPageInterpreter(resource_manager, text_converter)

    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)

    text = out_text.getvalue()

    # Close resources
    fp.close()
    text_converter.close()
    out_text.close()

    # Save extracted text to Markdown (.md) file
    with open(output_md_file, "w", encoding="utf-8") as md_file:
        md_file.write(text)

    print(f"Markdown file saved at: {output_md_file}")

# Example usage
pdf_path = "./knowledge/mitsubisi_electric.pdf"
md_path = "./knowledge/output.md"
get_pdf_file_content(pdf_path, md_path)


Markdown file saved at: ./knowledge/output.md


## Speech to text using Groq (wisper)

In [4]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm = ChatGroq(model_name="Llama3-8b-8192")

In [13]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq()

# Specify the path to the audio file
# filename = os.path.dirname("C:/Users/mayur/Desktop/datasmith ai") + "/sample_audio.mp3" # Replace with your audio file!
filename = "C:/Users/mayur/Desktop/datasmith ai/sample_audio.mp3"
# Open the audio file
with open(filename, "rb") as file:
    # Create a translation of the audio file
    translation = client.audio.translations.create(
      file=(filename, file.read()), # Required audio file
      model="whisper-large-v3", # Required model to use for translation
      prompt="Specify context or spelling",  # Optional
      response_format="json",  # Optional
      temperature=0.0  # Optional
    )
    # Print the translation text
    print(translation.text)

 Machine learning teach a computer how to perform a task without explicitly programming it to perform said task instead feed data into an algorithm to gradually improve outcomes with experience similar to how organic life learns the term was coined in 1959 by Arthur Samuel at IBM who is developing artificial intelligence that could play checkers half a century later and Predictive models are embedded in many of the products we use every day which perform two fundamental jobs One is to classify data like is there another car on the road or does this patient have cancer? The other is to make predictions about future outcomes like will the stock go up or which YouTube video do you want to watch next? The first step in the process is to acquire and clean up data Lots and lots of data the better the data represents the problem the better the results garbage in garbage out the data needs to have some kind of signal to be valuable to the algorithm for making predictions and data Data scientis

## Text To speech using gtts

In [ ]:
from gtts import gTTS
import io
def text_to_audio_buffer(text):
    tts = gTTS(text)

    # Store the audio in memory (without saving a file)
    audio_buffer = io.BytesIO()
    tts.write_to_fp(audio_buffer)
    audio_buffer.seek(0)  # Move to the beginning of the buffer

    return audio_buffer

## Text To speech using ElevenLabs

In [11]:
import os
from dotenv import load_dotenv
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs

# Load environment variables
load_dotenv()

ELEVENLABS_API_KEY = os.getenv("ELEVEN_API_KEY")

if not ELEVENLABS_API_KEY:
    raise ValueError("ELEVENLABS_API_KEY environment variable not set")

client = ElevenLabs(
    api_key=ELEVENLABS_API_KEY,
)

def text_to_speech_file(text: str) -> str:
    # Calling the text_to_speech conversion API with detailed parameters
    response = client.text_to_speech.convert(
        voice_id="pNInz6obpgDQGcFmaJgB",  # Adam pre-made voice
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_turbo_v2",  # use the turbo model for low latency, for other languages use the `eleven_multilingual_v2`
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
        ),
    )

    # Generating a unique file name for the output MP3 file
    save_file_path = "./output.mp3"
    # Writing the audio stream to the file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"A new audio file was saved successfully at {save_file_path}")

    # Return the path of the saved audio file
    return save_file_path


In [13]:
text = "Hello, I am an AI assistant designed to help you with your queries. How can I assist you today?"
file = text_to_speech_file(text)

ApiError: status_code: 401, body: {'detail': {'status': 'detected_unusual_activity', 'message': 'Unusual activity detected. Free Tier usage disabled. If you are using a proxy/VPN you might need to purchase a Paid Plan to not trigger our abuse detectors. Free Tier only works if users do not abuse it, for example by creating multiple free accounts. If we notice that many people try to abuse it, we will need to reconsider Free Tier altogether. \nPlease play fair and purchase any Paid Subscription to continue.'}}

## Text to speech using smallest.ai

In [ ]:
from smallest import Smallest
import os 

def text_to_speech_file(text):
    client = Smallest(api_key=os.getenv("SMALLEST_API_KEY"))
    client.synthesize(
        text,
        sample_rate=24000,
        speed=1.0,
        save_as="output.wav"
    )

    return "output.wav"




In [17]:
text = "Hello, I am an AI assistant designed to help you with your queries. How can I assist you today?"
file = text_to_speech_file(text)